In [1]:
%%bash
# Install the required Python 3 dependencies
python3 -m pip install kafka-python flask

In [2]:
from kafka import KafkaProducer

topic ="smoke-test"
producer = KafkaProducer(bootstrap_servers=['localhost:9092'])
# Note: The data you send must be binary
producer.send(topic, b"Hello World!").get(timeout=30)
# Note: The data you send must be binary
producer.send(topic, b"Foo").get(timeout=30)
producer.send(topic, b"Bar").get(timeout=30)

RecordMetadata(topic='smoke-test', partition=0, topic_partition=TopicPartition(topic='smoke-test', partition=0), offset=2, timestamp=1605548184318, log_start_offset=None, checksum=2322400209, serialized_key_size=-1, serialized_value_size=3, serialized_header_size=-1)

In [3]:
from IPython.display import display, clear_output
from time import sleep

import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0 pyspark-shell'
os.environ['SPARK_OPTS'] = "--driver-java-options=-Xms1024M --driver-java-options=-Xmx4096M --driver-java-options=-Dlog4j.logLevel=warn"


import pyspark 
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[2]').getOrCreate()
sc = spark.sparkContext

df = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers","localhost:9092")
    .option("subscribe", "smoke-test")
    .option("startingOffsets", "earliest")
    .load()
)

smoke_test_query = (
    df.writeStream                  # Create an output stream for the query
    .outputMode("append")           # Add each msg to end of "streaming table"
    .format("memory")               # Write output stream to Spark table
    .queryName("test_query") 
    .start()
)

# Wait until query runs for a bit
sleep(10)

# Display status of query
display(smoke_test_query.status)

# Show result in Jupyter Notebook
display(
    spark.table("test_query").toPandas()
)

# Stop query
smoke_test_query.stop()


{'message': 'Getting offsets from KafkaV2[Subscribe[smoke-test]]',
 'isDataAvailable': False,
 'isTriggerActive': True}

,key,value,topic,partition,offset,timestamp,timestampType
0,None,"[72, 101, 108, 108, 111, 32, 87, 111, 114, 108...",smoke-test,0,0,2020-11-16 17:36:24.289,0
1,None,"[70, 111, 111]",smoke-test,0,1,2020-11-16 17:36:24.316,0
2,None,"[66, 97, 114]",smoke-test,0,2,2020-11-16 17:36:24.318,0
